In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# Configuração do SparkSession com mais opções
spark = SparkSession.builder \
    .appName("DataFrame Example") \
    .master("local[*]") \
    .getOrCreate()

# Teste para verificar se a conexão está funcionando
print(f"Versão do Spark: {spark.version}")
print(f"URL do Master: {spark.sparkContext.master}")
print(f"Aplicativo: {spark.sparkContext.appName}")

# Crie um DataFrame simples para testar
df = spark.createDataFrame([(1, "teste"), (2, "exemplo")], ["id", "valor"])
df.show()

25/04/28 15:05:27 WARN Utils: Your hostname, guilhermerc-desktop resolves to a loopback address: 127.0.1.1; using 192.168.2.157 instead (on interface enp5s0)
25/04/28 15:05:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/28 15:05:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Versão do Spark: 3.5.5
URL do Master: local[*]
Aplicativo: DataFrame Example


+---+-------+
| id|  valor|
+---+-------+
|  1|  teste|
|  2|exemplo|
+---+-------+



In [2]:
# Caminho para o dataset TPC-DS
base_path = "/home/guilhermerc/Documentos/workspace/pyspark-playground/book_data/tpcds_10"

# Listar todos os diretórios (tabelas) no caminho
tables = []

# Como estamos executando de fora do contêiner, precisamos listar as tabelas de outra maneira
# Uma maneira é usando o código abaixo com o spark para listar as tabelas
from pyspark.sql.types import StringType

# Lista de tabelas conhecidas do TPC-DS
tpcds_tables = [
    "call_center", "catalog_page", "catalog_returns", "catalog_sales", 
    "customer", "customer_address", "customer_demographics", "date_dim",
    "household_demographics", "income_band", "inventory", "item",
    "promotion", "reason", "ship_mode", "store", "store_returns",
    "store_sales", "time_dim", "warehouse", "web_page", "web_returns",
    "web_sales", "web_site"
]

# Dicionário para armazenar os resultados
results = {}

print("Carregando tabelas para o catálogo e contando registros...")
print("-" * 60)

# Para cada tabela, carregue os dados e registre como tabela no catálogo
for table in tpcds_tables:
    table_path = f"{base_path}/{table}"
    
    # Tentativa de carregar o dataframe
    try:
        # Carregar a tabela como DataFrame
        df = spark.read.parquet(table_path)
        
        # Registrar como tabela temporária
        df.createOrReplaceTempView(table)
        
        # Contar os registros
        count = df.count()
        
        # Armazenar o resultado
        results[table] = count
        
        print(f"Tabela '{table}' carregada com sucesso: {count} registros")
    except Exception as e:
        print(f"Erro ao carregar tabela '{table}': {str(e)}")

# Exibir um resumo dos resultados
print("\nResumo das tabelas carregadas:")
print("-" * 60)
print("Tabela".ljust(30) + "| Registros")
print("-" * 60)

for table, count in sorted(results.items()):
    print(f"{table.ljust(30)}| {count}")

print("-" * 60)
print(f"Total de tabelas carregadas: {len(results)}")

# Listar as tabelas disponíveis no catálogo
print("\nTabelas disponíveis no catálogo:")
spark.catalog.listTables().show()

Carregando tabelas para o catálogo e contando registros...
------------------------------------------------------------


25/04/28 15:05:41 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
25/04/28 15:05:41 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


Tabela 'call_center' carregada com sucesso: 24 registros
Tabela 'catalog_page' carregada com sucesso: 12000 registros
Tabela 'catalog_returns' carregada com sucesso: 1439882 registros
Tabela 'catalog_sales' carregada com sucesso: 14400425 registros
Tabela 'customer' carregada com sucesso: 500000 registros
Tabela 'customer_address' carregada com sucesso: 250000 registros
Tabela 'customer_demographics' carregada com sucesso: 1920800 registros
Tabela 'date_dim' carregada com sucesso: 73049 registros
Tabela 'household_demographics' carregada com sucesso: 7200 registros
Tabela 'income_band' carregada com sucesso: 20 registros
Tabela 'inventory' carregada com sucesso: 133110000 registros
Tabela 'item' carregada com sucesso: 102000 registros
Tabela 'promotion' carregada com sucesso: 500 registros
Tabela 'reason' carregada com sucesso: 45 registros
Tabela 'ship_mode' carregada com sucesso: 20 registros
Tabela 'store' carregada com sucesso: 102 registros
Tabela 'store_returns' carregada com suc

AttributeError: 'list' object has no attribute 'show'